In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install transformers[torch] datasets torch peft trl==0.4.7 bitsandbytes accelerate cuda-python

In [ ]:
!zip -r ./fine_tuned_model.zip ./fine_tuned_model

In [ ]:
!unzip ./lora_model.zip

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Init imports and constants

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("HuggingFaceH4/no_robots", split="train_sft")

In [ ]:
# Loading model and tokenizer

quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

formatted_dataset = dataset.map(lambda row: {"formatted": tokenizer.apply_chat_template(row["messages"], tokenize=False, add_generation_prompt=False)})

In [ ]:
print(base_model.get_memory_footprint() / 1**9)

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

lora_model = get_peft_model(base_model, lora_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="training_output",
    num_train_epochs=1,
    logging_steps=100,
    save_steps=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    fp16=True
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=formatted_dataset,
    dataset_text_field="formatted",
    args=training_arguments
)

trainer.train()
trainer.save_model("./lora_model")

In [ ]:
# Testing original model
full_base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
)

text_gen = pipeline(
    "text-generation",
    model=BASE_MODEL,
)

messages = [
    {
        "role": "user",
        "content": "Generate a list of tags for a given quote: the world is you oyster",
    },
]

prompt = text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = text_gen(prompt)
print(result[0]["generated_text"])

In [ ]:
full_base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
)

lora_adapter = PeftModel.from_pretrained(
    full_base_model,
    "/content/lora_model"
)

fine_tuned_model = lora_adapter.merge_and_unload()

fine_tuned_model.save_pretrained("/content/fine_tuned_model")
tokenizer.save_pretrained("/content/fine_tuned_model")

In [ ]:
# Testing fine tuned model

fine_text_gen = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer
)

messages = [
    {
        "role": "user",
        "content": "Tell me the wrong answer to this question: what is 1+1?",
    },
]

prompt = fine_text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = fine_text_gen(prompt)
print(result[0]["generated_text"])